# Extra Methods in Fleche

When you decorate a function with `@fleche`, it's not just a cached version of the original function. It also gains several helper methods that allow you to interact with the cache and the function's metadata directly.

In [1]:
from fleche import fleche
import time

In [2]:
@fleche
def slow_add(a, b):
    """Adds two numbers slowly."""
    time.sleep(1)
    return a + b

## 1. `.call(*args, **kwargs)`

The `.call` method returns a `Call` object. This object captures all the information about a potential call to the function, including the function name, arguments, and metadata, without actually executing the function.

In [3]:
call_info = slow_add.call(10, b=20)
print(f"Function Name: {call_info.name}")
print(f"Arguments: {call_info.args}")
print(f"Keyword Arguments: {call_info.kwargs}")
print(f"Call Object: {call_info}")

Function Name: slow_add
Arguments: (10,)
Keyword Arguments: {'b': 20}
Call Object: Call(name='slow_add', args=(10,), kwargs={'b': 20}, metadata={}, module='__main__', version=None, result=None)


## 2. `.digest(*args, **kwargs)`

The `.digest` method returns the unique cache key (a SHA-256 hash) that `fleche` uses to identify this specific call in the cache. This is derived from the function name, arguments, and other metadata.

In [4]:
key = slow_add.digest(10, b=20)
print(f"Cache Key: {key}")

Cache Key: ebe3dae6bdd18eb9b888a662220433d2fa08c33a7540a0767b4da5cb3e9dcf08


## 3. `.contains(*args, **kwargs)`

You can use `.contains` to check if a result for a specific set of arguments is already present in the cache, without triggering the function execution.

In [5]:
print(f"Is (10, 20) in cache? {slow_add.contains(10, b=20)}")

# Now we run it to populate the cache
slow_add(10, b=20)

print(f"Is (10, 20) in cache now? {slow_add.contains(10, b=20)}")

Is (10, 20) in cache? False


Is (10, 20) in cache now? True


## 4. `.load(*args, **kwargs)`

The `.load` method allows you to explicitly retrieve a result from the cache. If the result is not in the cache, it will raise a `KeyError` instead of executing the function.

In [6]:
result = slow_add.load(10, b=20)
print(f"Loaded result: {result}")

try:
    slow_add.load(1, 2)
except KeyError:
    print("Result for (1, 2) not in cache.")

Loaded result: 30
Result for (1, 2) not in cache.


## 5. `.__wrapped__`

Because `fleche` uses `functools.wraps`, the original undecorated function is always available via the `.__wrapped__` attribute. This is useful if you want to bypass the cache entirely.

In [7]:
start = time.time()
res = slow_add.__wrapped__(10, 20)
end = time.time()
print(f"Result: {res} (took {end - start:.2f} seconds bypassing cache)")

Result: 30 (took 1.00 seconds bypassing cache)
